Сайт https://nashformat.ua/catalog/top-30-knyzhok-2017-roku

1. Витягти з сайту: назва, автор, рік видання, рейтинг і т.д.
2. Одну таблицю в бд
3. вивести назви всіх книжок користувач вводить книгу (цифру)- зберегти її палітурку у jpeg

In [135]:
# 1. Витягти з сайту: назва, автор, рік видання, рейтинг і т.д.


# + в наявності/не в наявності 


import requests
from bs4 import BeautifulSoup
import datetime



all_books = {}
i = 1 #key in dict

r = requests.get('https://nashformat.ua/catalog/top-30-knyzhok-2017-roku')
soup = BeautifulSoup(r.text, 'html.parser')
all_info = soup.find_all('div', class_='product-list_content')
print(all_info[0])
for el in all_info:
    title = el.find('a', class_='h3').get('title')
    book_link = el.find('a', class_='h3').get('href')
    author = el.find('h4').text.lstrip('/n').lstrip().rstrip()
    rating_tag = el.find('div', class_='product_rating_js').find_all('span')[2].text
    rating = float(rating_tag.lstrip('Рейтинг: '))
    publication_year_tag = el.find('span', class_='color_dark-gray').text
    publication_year = int(publication_year_tag.lstrip('Рік видання: '))
    price = float(el.find('span', class_ = 'cost-count').text)
    all_books[i] = {'title':title, 'link':book_link, 'author':author, 'rating':rating,
                    'publ_year':publication_year, 'price':price
                   }
    i += 1
print(all_books[1]['author'])

<div class="product-list_content"> <div class="product-list_title"> <h5><a class="h3" data-product="41879" href="products/kazka-pro-dobromola-920560" title="Казка про Добромола">Казка про Добромола</a></h5> </div> <h4 class="product-list_author h4">
                    Олександр Турчинов                </h4> <div class="product_rating_js"> <span class="rating_starOff"><span class="rating_starOn" style="width:84.6px;"></span></span> <span data-language="product_rating">Рейтинг: 4.7</span> <span>(8 відгуків)</span> </div> <div class="row product-list_features"> <span class="color_dark-gray span_small">Рік видання: 2020</span> <span class="color_dark-gray span_small">Палітурка: тверда</span> <span class="color_green">
                                                    Книга в наявності
                                            </span> </div> <div class="row product-list_desc">
                 Книга для свідомих та розумних читачів, які не голосують за казкових персонажів. 
 ТЕМАТИКА 


In [103]:
# 2. Одну таблицю в бд

import sqlite3


conn = sqlite3.connect('Books.db')
curs = conn.cursor()

sql_text = '''
    CREATE TABLE Book (
        name TEXT NOT NULL,
        author TEXT NOT NULL,
        publ_year INTEGER NOT NULL,
        rating REAL,
        price REAL,
        link TEXT NOT NULL
    )
'''

curs.execute(sql_text)
conn.close()

In [104]:
conn = sqlite3.connect('Books.db')
curs = conn.cursor()

sql_insert = '''
    INSERT INTO Book (name, author, publ_year, rating, price, link) VALUES (?,?,?,?,?,?)
'''

for el in all_books.items():
    current_el = el[1]
    curs.execute(sql_insert, (current_el['title'],
                              current_el['author'],
                               current_el['publ_year'],
                             current_el['rating'],
                             current_el['price'],
                             current_el['link']))
conn.commit()

conn.close()

In [168]:
#3. вивести назви всіх книжок користувач вводить книгу (цифру)- зберегти її палітурку у jpeg

conn = sqlite3.connect('Books.db')
curs = conn.cursor()

sql_get_books = '''
    SELECT name, author FROM Book
'''

books_and_authors = curs.execute(sql_get_books).fetchall()

i = 1
for el in books_and_authors:
    print(i, el[0], el[1])
    i += 1

selected_book = int(input('Введіть обрану книгу: '))
print()

book_to_search = books_and_authors[selected_book-1][0]

sql_find_book = '''
 SELECT * FROM Book
 WHERE name = (?)
'''

found_book = curs.execute(sql_find_book, (book_to_search,)).fetchall()[0]
print('Інформація про обрану книгу:\n', found_book)
conn.close()
link = 'https://nashformat.ua/'+found_book[5]


r = requests.get(link)
soup = BeautifulSoup(r.text, 'html.parser')

img = soup.find_all('a', class_="gallery fn-zoom btn-block img-block")[0].get('href')
r = requests.get(img)
img_binary = r.content
with open ('image.jpeg', 'wb') as f:
     f.write(img_binary)

1 Казка про Добромола Олександр Турчинов
2 Справа Василя Стуса. Збірка документів з архіву колишнього КДБ УРСР Вахтанг Кіпіані
3 Сліди на дорозі Валерій Ананьєв
4 Амадока Софія Андрухович
5 Купи собі той довбаний букет: та інші способи зібратися докупи від тієї, кому вдалось Тара Шустер
6 Нація овочів? Як інформація змінює мислення і поведінку українців Оксана Мороз
7 Людина розумна Ювал Ной Харарі
8 Про що я говорю, коли говорю про біг Харукі Муракамі
9 21 урок для 21 століття Ювал Ной Харарі
10 Було-стало. Нові зміни в правописі Олександр Авраменко
11 Зваблення їжею: 70 рецептів, які захочеться готувати Євген Клопотенко
12 Доця Тамара Горіха Зерня
13 Місто дівчат Елізабет Ґілберт
14 Пиши, скорочуй. Як створити дієвий текст Людмила Саричева, Максим Ільяхов
15 Характерник Василь Шкляр
16 Війна за реальність. Як перемагати у світі фейків, правд і спільнот Дмитро Кулеба
17 7 звичок надзвичайно ефективних людей Стівен Р. Кові
18 Ворошиловград. Дике поле. Захищай своє Сергій Жадан
19 Психо

In [151]:
type(img)

bs4.element.ResultSet